In [21]:
# This is practice for convoluted neural networks
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Step 1 - Preparing the data for the model
from datasets import load_dataset

ds = load_dataset("ylecun/mnist")
train_loader = torch.utils.data.DataLoader(ds, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(ds, batch_size=64, shuffle=False)

In [ ]:
# Step 2 - Creating the Validator
def validate(model, train_loader, val_loader):
    # Loops through the training and validation sets
    for name, loader in [('train', train_loader),('val', val_loader)]:
        correct = 0
        total = 0

        # Disable gradient computation since we are only doing inference
        with torch.no_grad()

            # Loops through each image in the train + val sets
            for imgs, labels in loader:

            # Sends the images and labels to the appropriate device (GPU or CPU)
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)

            # Passes the images through the model to get predicted values
            outputs = model(imgs)

            # Retrieves the index of Maximum probability for each vector of predictions
            _, predicted = torch.max(outputs, dim=1)

            # Updates the running total of samples and correct predictions
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

        # Calculate and prints the accuracy for the current dataset
        print("Accuracy {}: {:.2f}".format(name , correct / total))            

In [24]:
# Step 3 - Creating the Residual Block in the Neural Network
class ResBlock(nn.Module):

    # Initializing the class
    def __init__(self, n_chans):
        # Initializing the parent class to ensure proper setup
        super(ResBlock, self).__init__()

        # Creating the convolution that will be applied to the input
        self.conv = nn.Conv2d(1,n_chans,kernel_size=2,padding=1,bias=False)

        # Batch normalization to normalize the activations
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)

        # Initializing the weights of the convolutional layer with Kaiming Normal initialization
        # This makes weights follow a normal distribution with a mean of 0,
        # and a standard deviation that depends on the number of input units
        torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')

        # Initializing batch normalization parameters
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    # Defining the forward pass of the models, which dictates how data moves through the network
    def forward(self, x):
        # Applies the convolutional layer to the input
        out = self.conv(x)

        # Applies the batch normalization layer to the input
        out = self.batch_norm(out)

        # Applies the ReLU activation function to introduce non-linearity
        out = torch.relu(out)

        # Adds the input to the output (residual connection)
        # This helps to mitigate the vanishing gradient problem and allows earlier gradients
        # to impact the output of the model
        return out + x

In [26]:
# Step 4 - Creating the residual model
class ResNet(nn.Module):
    # Initializing the variables in the class
    def __init__(self, n_chans1=28, n_blocks=10):
        
        # Initializing the parent class
        super().__init__()

        # Initializes the width of each layer
        self.n_chans1 = n_chans1

        # Creating the convolution to be applied to each layer
        self.conv = nn.Conv2d(1, n_chans1, kernel_size=1, padding=1, bias=False)

        # Creating the Residual Blocks that will be stacked for the input to flow through
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))

        # Creating the fully connected networks to tie it all together
        # Primarily to restrict data to feed into output (10, 0-9)
        self.fc1 = nn.Linear(8 * 8 * self.n_chans1, 28)
        self.fc2 = nn.Linear(28, 10)

    # Creating the forward pass for how the data goes through the network
    def forward(self, x):
        # Applies the Functional version of MaxPool to identify structures, ReLU activation to introduce non-linearity, and runs the convolution
        out = F.max_pool2d(torch.relu(self.conv(x)),2)

        # Sends the data through the ResBlocks
        out = self.resblocks(out)

        # Applies the max pool to identify further structures in the image
        out = F.max_pool2d(out, 2)

        # Flattens out the network from 2d to 1d as fully connected networks require 1d
        out = out.view(-1, 8 * 8 * self.n_chans1)

        # Starts sending the data through the finally activation function and fully connected networks
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out


In [18]:
img = ds['train'][0]
ds.shape

{'train': (60000, 2), 'test': (10000, 2)}

In [19]:
img['image']